In [20]:
import numpy as np
import trimesh
import tqdm 

In [21]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
    v1 = vertices[faces[:, 0]]
    v2 = vertices[faces[:, 1]]
    v3 = vertices[faces[:, 2]]
    a = np.linalg.norm(v2 - v1, axis=1)
    b = np.linalg.norm(v3 - v2, axis=1)
    c = np.linalg.norm(v1 - v3, axis=1)
    p = (a + b + c) / 2
    area = np.sqrt(p * (p - a) * (p - b) * (p - c))
    prob = area / np.sum(area)
    face_idx = np.random.choice(faces.shape[0], size=sample_num, p=prob)
    sampled_faces = faces[face_idx]
    sampled_v1 = vertices[sampled_faces[:, 0]]
    sampled_v2 = vertices[sampled_faces[:, 1]]
    sampled_v3 = vertices[sampled_faces[:, 2]]
    u = np.random.rand(sample_num)
    v = np.random.rand(sample_num)
    sqrt_u = np.sqrt(u)
    uniform_pc = ( (1 - sqrt_u)[:, None] * sampled_v1 +
                   (sqrt_u * (1 - v))[:, None] * sampled_v2 +
                   (sqrt_u * v)[:, None] * sampled_v3 )
    
    return area, prob, uniform_pc
        

In [22]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------
    n_points = pc.shape[0]
    results = np.zeros((sample_num, pc.shape[1]))
    dis = np.full(n_points, np.inf)
    ini_idx = np.random.randint(n_points)
    results[0] = pc[ini_idx]
    
    for i in range(1, sample_num):
        dis = np.minimum(dis, np.linalg.norm(pc - results[i-1], axis=1))
        farthest_idx = np.argmax(dis)
        results[i] = pc[farthest_idx]
        
    return results

In [23]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [24]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

In [25]:
# task 3: metrics

from earthmover.earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

CD_mean = 0
CD_var = 0
EMD_mean = 0
EMD_var = 0

def onewayChamferDistance(A, B):
    _A = A[:, np.newaxis]
    _B = B[np.newaxis]
    distance = np.sqrt(np.sum((_A - _B) ** 2, axis = -1))
    return np.array([np.mean(np.min(distance, axis=1), axis=0)])
def chamferDistance(A, B):
    return (onewayChamferDistance(A, B) + onewayChamferDistance(B, A)) / 2

CD = []
EMD = []

for i in range(5):
    _, _, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)
    fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)
    CD.append(chamferDistance(uniform_pc, fps_pc))
    EMD.append(earthmover_distance([tuple(x) for x in uniform_pc.tolist()], [tuple(x) for x in fps_pc.tolist()]))
CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)
# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

load c:\Users\31006\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\31006\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\31006\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\31006\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\31006\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\31006\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\31006\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\ortools.dll...
move 0.001953125 dirt from (27.88129696319188, 18.266423385709547, 20.243106227831397) to (27.670105403693185, 15.083783558355531, 20.286449559610052) for a cost of 0.006230339190301998
move 0.001953125 dirt from (22.621949603659605, 22.072914704